# Redes neurais artificiais

In [58]:
import h5py
import torch
import torch.nn as nn
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from skorch import NeuralNetBinaryClassifier, NeuralNetClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from torch.autograd import Variable

## Redes MPL

In [59]:
data = pd.read_csv('data/credtrain.txt', sep='\t', header=None)
data_test = pd.read_csv('data/credtest.txt', sep='\t', header=None)

X = np.array(data.iloc[:, :-1]).astype(np.float32)
y = np.array(data.iloc[:, -1]).astype(np.float32)

X_test = np.array(data_test.iloc[:, :-1]).astype(np.float32)
y_test = np.array(data_test.iloc[:, -1])

ss = StandardScaler()
X_norm = ss.fit_transform(X)
X_test_norm = ss.fit_transform(X_test)

In [60]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(11, 64)
        self.fc2 = torch.nn.Linear(64, 1)
        
    def forward(self, x, **kwargs):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

net = NeuralNetBinaryClassifier(Net, max_epochs=200, lr=0.01, criterion=torch.nn.BCEWithLogitsLoss, verbose=0)
net.fit(X_norm, y)
y_predict = net.predict(X_test_norm)
print(classification_report(y_predict, y_test))

              precision    recall  f1-score   support

           0       0.97      0.86      0.91       347
           1       0.82      0.96      0.88       230

    accuracy                           0.90       577
   macro avg       0.89      0.91      0.90       577
weighted avg       0.91      0.90      0.90       577



## Redes convolucionais

### MPL

In [61]:
def load_dataset():
    train_dataset = h5py.File('data/train_catvnoncat.h5', 'r' )
    train_set_x_orig = np.array(train_dataset['train_set_x'][:]).astype(np.float32) #your train set features
    train_set_y_orig = np.array(train_dataset['train_set_y'][:]) #your train set labels

    test_dataset = h5py.File('data/test_catvnoncat.h5', 'r' )
    test_set_x_orig = np.array(test_dataset['test_set_x'][:]).astype(np.float32) #your test set features
    test_set_y_orig = np.array(test_dataset['test_set_y'][:]) #your test set labels

    classes = np.array(test_dataset['list_classes'][:]) #the list of classes

    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

X_train, y_train, X_test, y_test, classes = load_dataset()

X_train_flatten = X_train.reshape(X_train.shape[0], -1) / 255
X_test_flatten = X_test.reshape(X_test.shape[0], -1) / 255

In [64]:
class NetSoftmax(torch.nn.Module):
    def __init__(self):
        super(NetSoftmax, self).__init__()
        self.fc1 = torch.nn.Linear(64 * 64 * 3, 64)
        self.fc2 = torch.nn.Linear(64, 2)
        
    def forward(self, x, **kwargs):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
net_softmax = NeuralNetClassifier(NetSoftmax, max_epochs=250, lr=0.01, criterion=torch.nn.CrossEntropyLoss, verbose=0)
net_softmax.fit(X_train_flatten, y_train)

y_predict = net_softmax.predict(X_test_flatten)

print(classification_report(y_predict, y_test, zero_division=1))

              precision    recall  f1-score   support

           0       0.65      0.55      0.59        20
           1       0.73      0.80      0.76        30

    accuracy                           0.70        50
   macro avg       0.69      0.68      0.68        50
weighted avg       0.70      0.70      0.69        50

